In [59]:
your_username = 'somayeh91'


# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os
import sys

# sys.path.append('/home/idies/Storage/'+your_usernamer+'/persistent/sims_maf_contrib/')
# import lsst.sim.maf moduels modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles
print(os.getcwd())
# import convenience functions
from opsimUtils import *

dbDir = '/home/idies/workspace/lsst_cadence/FBS_1.6/'
outDir = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/db_results_3_4_2021/'.format(your_username)

resultsDb = db.ResultsDb(outDir=outDir)
if not os.path.exists(os.path.abspath(outDir)):
    os.mkdir(os.path.abspath(outDir))
    
opSimDbs, resultDbs = connect_dbs(dbDir, outDir)

/home/idies/workspace/Storage/somayeh91/persistent/Rubin_Microlensing_metrics


In [60]:
DBPATH_V1_5 = dbDir
COLS = 'night, observationStartMJD, fieldRA, fieldDec, filter, proposalId, note'
NSIDE = 64

In [61]:
def GP_slicer_selector (dbname, COLS = 'night, observationStartMJD, fieldRA, fieldDec, filter, proposalId, note',\
                       NSIDE = 64, DBPATH_V1_5 = dbDir):
    #dbname =  'footprint_add_mag_cloudsv1.5_10yrs.db'

    # get some data from opsim database
    df_opsim = get_cols(dbfile=DBPATH_V1_5 + dbname, cols=COLS)

    # get healpix Ids
    df_opsim = get_pixId(df_opsim, nside=NSIDE)

    # in galactic coordinates
    df_opsim['fieldGl'], df_opsim['fieldGb'] = radec2gal(ra=df_opsim['fieldRA'].values, dec=df_opsim['fieldDec'].values)

    # define the shape of galactic plane
    peakDeg=15.
    taperDeg=90.
    constWidth=.5

    # get the latitude of galaxy region
    df_opsim['galaxyb'] = getAvoidanceLatitudes(df_opsim['fieldGl'].values, peakDeg, taperDeg, constWidth)

    # check if a field is ingalaxy
    df_opsim['ingalaxy'] = ( df_opsim['fieldGb'] > -df_opsim['galaxyb'] ) & ( df_opsim['fieldGb'] < df_opsim['galaxyb'] ) 

    # get pixId in galactic plane
    ipix_GP = df_opsim.pixId[df_opsim.ingalaxy].unique()
    
    ra, dec = pix2radec(nside=64, ipix=ipix_GP)
    slicer = slicers.UserPointsSlicer(ra, dec, latLonDeg=True, badval=0)


    
    return slicer




In [62]:
# !pip install sympy

In [76]:
import glob 

dbruns_selected = glob.glob(dbDir+"even*")
dbruns_selected.sort()

In [77]:
dbruns_selected

['/home/idies/workspace/lsst_cadence/FBS_1.6/even_filters_alt_g_v1.6_10yrs.db',
 '/home/idies/workspace/lsst_cadence/FBS_1.6/even_filters_altv1.6_10yrs.db',
 '/home/idies/workspace/lsst_cadence/FBS_1.6/even_filters_g_v1.6_10yrs.db',
 '/home/idies/workspace/lsst_cadence/FBS_1.6/even_filtersv1.6_10yrs.db']

In [74]:
# pip install sympy

In [66]:
# WFIRST_OW = [['2025-04-25', '2025-07-07'],['2026-04-25', '2026-07-07'],['2025-04-25', '2026-07-07']]

In [67]:
LSST_filters = ["g","r","i","z"]
import sys
sys.path.insert(0, '../sims_maf_contrib/')

summaryMetrics = [metrics.MedianMetric(), metrics.MeanMetric(), metrics.MinMetric(), metrics.MaxMetric(), metrics.RmsMetric(),\
                 metrics.SumMetric(), metrics.CountMetric()]

from mafContrib import StarCountMetric,CampaignLengthMetric, numObsInSurveyTimeOverlapMetric, IntervalsBetweenObsMetric
# sys.path.insert(0, '../sims_maf_contrib/mafContrib')
# from numObsInSurveyTimeOverlapMetric import numObsInSurveyTimeOverlapMetric
starcount= StarCountMetric()
campaign =  CampaignLengthMetric()
# numobs = numObsInSurveyTimeOverlapMetric(WFIRST_OW)
# intervalObs = IntervalsBetweenObsMetric(WFIRST_OW, Stat = 'median')
cadence = metrics.cadenceMetrics.NRevisitsMetric()
uniformity = metrics.UniformityMetric()
#List of metrics

Microlensing_metrics = [ starcount, campaign, cadence, uniformity]

names_metric = ['starcount','campaignlength','nrevisits', 'uniformity']
# names_metric = ['campaignlength','nrevisits', 'uniformity']



In [10]:
# #Slicer
# # slicer1 = slicers.HealpixSlicer(nside=64)
# slicer = GP_slicer_selector()
# #List of bundles
# Microlensing_bundles = []
# Names_metric = []
# for ind_fil,fil in enumerate(LSST_filters):
    
#     if fil == 'u':
#         continue
    
#     constraint1 = 'filter = "'+fil+'"'
#     print(constraint1)
#     for ind_met,metri in enumerate(Microlensing_metrics):
        
#         microlensing_metric = metricBundles.MetricBundle(metri, slicer, constraint1, summaryMetrics=summaryMetrics)
#         Microlensing_bundles.append(microlensing_metric)
#         Names_metric.append(names_metric[ind_met]+'_'+fil)

In [35]:
# If running on GP only, you must make a bundle for each opsim run individually

def bundle_creator(slicer, LSST_filters):
    #List of bundles
    Microlensing_bundles = []
    Names_metric = []
#     for ind_fil,fil in enumerate(LSST_filters):

#         if fil == 'u':
#             continue

#         constraint1 = 'filter = "'+fil+'"'
#         print(constraint1)
    for ind_met,metri in enumerate(Microlensing_metrics):

        microlensing_metric = metricBundles.MetricBundle(metri, slicer, summaryMetrics=summaryMetrics)
        Microlensing_bundles.append(microlensing_metric)
        Names_metric.append(names_metric[ind_met])
    return Microlensing_bundles, Names_metric

In [36]:

# # create a bundleDict for all metrics to run on each opSim

metricDataPath = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/metric_data_3_4_2021/'.format(your_username)

In [78]:
# metricDataPath = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/output_10_26_2020/'.format(your_username)

# below I am only going to run metrics on the first five opsims
LSST_filters = ["u","g","r","i","z","y"]
for run in dbruns_selected:
    print (run)
    run_name = run.split('1.6/')[1].split('.d')[0]
    print(run_name)
    slicer = GP_slicer_selector(run_name+'.db')
    Microlensing_bundles, Names_metric = bundle_creator(slicer, LSST_filters)
    bundleDict = {Names_metric[i]:Microlensing_bundles[i] for i in range(len(Names_metric))}
    
    # must set run name for each opSim to store metric data into
    # separate files
#     numobs.setRunName(run_name)
#     intervalObs.setRunName(run_name)
#     starcount.setRunName(run_name)
#     campaignlength.setRunName(run_name)
#     nrevisits.setRunName(run_name)
#     uniformity.setRunName(run_name)
    metricGroup = metricBundles.MetricBundleGroup(bundleDict,\
                    opSimDbs[run_name], metricDataPath, resultDbs[run_name])
    metricGroup.runAll()
print ('done')

/home/idies/workspace/lsst_cadence/FBS_1.6/even_filters_alt_g_v1.6_10yrs.db
even_filters_alt_g_v1.6_10yrs
connect to  /home/idies/workspace/lsst_cadence/FBS_1.6/even_filters_alt_g_v1.6_10yrs.db 
 SELECT night, observationStartMJD, fieldRA, fieldDec, filter, proposalId, note FROM SummaryAllProps
Querying database SummaryAllProps with no constraint for columns ['observationStartMJD', 'fieldDec', 'fieldRA'].
Found 2235956 visits
Running:  ['starcount', 'campaignlength', 'nrevisits', 'uniformity']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
/home/idies/workspace/lsst_cadence/FBS_1.6/even_filters_altv1.6_10yrs.db
even_filters_altv1.6_10yrs
connect to  /home/idies/workspace/lsst_cadence/FBS_1.6/even_filters_altv1.6_10yrs.db 
 SELECT night, observationStartMJD, fieldRA, fieldDec, filter, proposalId, note FROM SummaryAllProps
Querying database SummaryAllProps with no constraint for columns ['observationStartMJD', 'fieldDec', 'fieldRA'].
Found 223

In [14]:
def getAvoidanceLatitudes(galL, peakDeg=10., taperDeg=80., constWidth=5.):
    
    """Returns the (positive) GP avoidance region for input galactic longitude
    Input:
        galL: in degree
        peakDeg: galaxy peak size
        taperDeg: half width, in degree
        constWidth: in degree
    return 
        val: the positive galaxy b, in degree
    """  
    # The following is adapted from spatialPlotters.py in sims_maf, method _plot_mwZone
    
    # astropy uses 0 <= glon <= 2pi, so we shift the input values accordingly.
    galL = np.radians(galL)
    galL_use = np.copy(galL)
    gSec = np.where(galL_use > np.pi)
    galL_use[gSec] -= 2.0 * np.pi
    
    peakWidth=np.radians(peakDeg)
    taperLength=np.radians(taperDeg)
    val = peakWidth * np.cos(galL_use / taperLength * np.pi / 2.)
    
    # Remove the cosine peak at anticenter
    gFar = np.where(np.abs(galL_use) > taperLength)[0]
    val[gFar] = 0.

    val += np.radians(constWidth)
    
    return np.rad2deg(val)

In [15]:
#  get columns from databases
def get_cols(dbfile, cols='night', lim=None):
    """
    cols = 'night, observationStartMJD, fieldRA, fieldDec, filter, proposalId, note'
    connect to database
    Returns: a pandas DataFrame
    
    Examples:
    
    cols = 'night, observationStartMJD, fieldRA, filter, fieldDec, proposalId, note, '

    df = get_cols(dbfile=dbpath + db, cols=cols, lim='night<300')
   
    """
    import sqlite3
    import pandas as pd
    connection = sqlite3.connect(dbfile)
    cursor = connection.cursor()
    
    if lim!=None:
        sqlstr = "SELECT {} FROM SummaryAllProps where {}".format(cols, lim)
    else:
        sqlstr = "SELECT {} FROM SummaryAllProps".format(cols)

    print('connect to ', dbfile, '\n', sqlstr)
    cursor.execute(sqlstr)
    data = cursor.fetchall()
    
    # convert data to dataframe
    data_df = pd.DataFrame(data, columns=cols.split(', '))
    
    # close connection
    connection.close()
    
    return data_df

def get_pixId(df, nside=16):
    '''get pixId for each field'''
    pixId = RADec2pix(nside, ra=df['fieldRA'].values, dec=df['fieldDec'].values)
    df['pixId'] = pixId
    return df

def pix2radec(nside, ipix, degree=True):
    """from ipix to ra/dec"""
    
    theta, phi = hp.pix2ang(nside, ipix)
    ra = phi
    dec = np.pi/2. - theta
    if degree:
        ra = np.rad2deg(ra)
        dec = np.rad2deg(dec)
    return ra, dec

def RADec2pix(nside, ra, dec, degree=True):
    """
    Calculate the nearest healpixel ID of an RA/Dec array, assuming nside.

    Parameters
    ----------
    nside : int
        The nside value of the healpix grid.
    ra : numpy.ndarray
        The RA values to be converted to healpix ids, in degree by default.
    dec : numpy.ndarray
        The Dec values to be converted to healpix ids, in degree by default.

    Returns
    -------
    numpy.ndarray
        The healpix ids.
    """
    if degree:
        ra = np.radians(ra) # change to radians
        dec = np.radians(dec)
    
    lat = np.pi/2. - dec
    hpid = hp.ang2pix(nside, lat, ra )
    return hpid

def radec2xyz(ra, dec):
    '''convert ra/dec to xyz'''
    theta = 0.5 * np.pi - np.deg2rad(dec)
    phi = np.deg2rad(ra)
    xyz = hp.ang2vec(theta, phi)
    
    return xyz

def radec2gal(ra, dec):
    '''convert from ra/dec to galactic l/b'''
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    c = SkyCoord(ra=ra, dec=dec, 
                 unit=(u.degree, u.degree))
    
    l = c.galactic.l.degree
    b = c.galactic.b.degree
    return l, b

def gal2radec(gl, gb):
    '''convert gl/gb to ra/dec'''
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    c = SkyCoord(l=gl, b=gb, frame="galactic",
                unit=(u.degree, u.degree))
    ra = c.icrs.ra.degree
    dec = c.icrs.dec.degree
    return ra, dec


# plot function
# plot skymap

def plot_mwd(ax, RA, Dec, c, org=0, title='Mollweide projection', projection='mollweide', vmax=None, colorbar=True, **kwargs):
    ''' RA, Dec are arrays of the same length.
    RA takes values in [0,360), Dec in [-90,90],
    which represent angles in degrees.
    org is the origin of the plot, 0 or a multiple of 30 degrees in [0,360).
    title is the title of the figure.
    projection is the kind of projection: 'mollweide', 'aitoff', 'hammer', 'lambert'
    
    Examples:
    >>> fig, ax = plt.subplots(1, 1, figsize=(8, 8), 
                            subplot_kw={'projection': 'mollweide'}); 
        plot_mwd(ax, x, y, c, title='') 
        
    '''
    import numpy as np
    import matplotlib.pyplot as plt

    if vmax is None:
        vmax = c.max()
    
    x = np.remainder(RA+360-org,360) # shift RA values
    ind = x>180
    x[ind] -=360    # scale conversion to [-180, 180]
    x=-x    # reverse the scale: East to the left
    tick_labels = np.array([150, 120, 90, 60, 30, 0, 330, 300, 270, 240, 210])
    tick_labels = np.remainder(tick_labels+360+org,360)
    # fig = plt.figure(figsize=(10, 5))
    # ax = fig.add_subplot(111, projection=projection, facecolor ='LightCyan')
    # ax = fig.add_subplot(111, projection=projection, facecolor ='honeydew')
    sc = ax.scatter(np.radians(x), np.radians(Dec), c=c, marker='o', vmin=0, vmax=vmax, **kwargs)  # convert degrees to radians
    # ax.set_xticklabels(tick_labels)     # we add the scale on the x axis
    ax.set_title(title)
    ax.title.set_fontsize(15)
    ax.set_xlabel("RA")
    ax.xaxis.label.set_fontsize(12)
    ax.set_ylabel("Dec")
    ax.yaxis.label.set_fontsize(12)
    ax.grid(True)
    if colorbar:
        cbar = plt.colorbar(sc, ax=ax, orientation="horizontal")
    #cbar.set_clim(-2.0, 2.0)

def plot_filter_Nv(df, coord='C', figsize= (12, 6), colorbar=True, vmax=None, **kwargs):
    """plot number of visits in 6 filters"""
    
    fig, axs = plt.subplots(2, 3, figsize=figsize,
                            subplot_kw={'projection': 'mollweide'}); 
    axs = axs.ravel()
    FILTERS = ['u', 'g', 'r', 'i', 'z', 'y']
    for i, f in enumerate(FILTERS):
        
        df_f = df[ df['filter']==f ]
        if coord=='G':
            x = df_f['fieldGl'].values
            y = df_f['fieldGb'].values       
        else:
            x = df_f['fieldRA'].values
            y = df_f['fieldDec'].values
        c = df_f['Nv'].values
        if len(c)!=0:
            plot_mwd(axs[i], x, y, c, title=f, colorbar=colorbar, vmax=vmax, **kwargs)
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
        axs[i].set_title(f)

